In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer, CustomSchedule
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import construct_hamiltonian, get_ground_state_for_g, get_theta_v, get_theta_w, get_theta_r

In [2]:
generator_layers = 2
discriminator_layers = 2
data_bus_size = 3

In [3]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

AssertionError: 

In [ ]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [ ]:
pure_gen = gen.copy()
gen.append([disc])

In [ ]:
pure_real = real.copy()
real.append([disc])

In [ ]:
print("REAL GROUND STATE")
print(pure_real)

In [ ]:
print("GENERATOR")
print(pure_gen)

In [ ]:
print("DISCRIMINATOR")
print(disc)

In [10]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 21) + \
                   np.random.normal(scale=eps, size=(22,))
init_disc_weights = np.random.normal(size=(28,))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [11]:
learning_rate = CustomSchedule()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [12]:
g_provider = lambda: 0

In [13]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  use_analytical_expectation=True)

In [ ]:
epochs = 50
disc_iteration = 100
gen_iteration = 2
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration)

Instructions for updating:
Use fn_output_signature instead
Epoch 0: cost = [[0.2982991]]
Prob(real classified as real):  [[0.4976718]]
Prob(fake classified as real):  [[0.7959709]]
Epoch 0: cost = [[-0.7959709]]
Prob(fake classified as real):  [[0.7959709]]
Discriminator cost:  [[0.2982991]]
Generator weights: <tf.Variable 'Variable:0' shape=(22,) dtype=float32, numpy=
array([-10.1538925,  -5.8757625,  -5.8703175,   3.001207 , -13.019004 ,
       -12.693356 ,   1.9784341, -13.137627 , -13.786987 , -13.515038 ,
       -13.311686 , -12.854742 ,   2.3864002,   5.4604774,  -5.8757343,
       -12.866825 , -13.503126 ,  -5.8818154,  -5.876974 ,  -5.8876247,
       -13.543596 , -13.511531 ], dtype=float32)>
Discriminator weights <tf.Variable 'Variable:0' shape=(28,) dtype=float32, numpy=
array([  28.310925  ,   -1.3881841 ,   -0.61370814, -102.06886   ,
        -83.644295  ,  -14.386199  ,   50.432297  ,   52.792294  ,
         82.30965   ,  -98.54453   ,   94.65001   ,   32.676636  ,
       

In [28]:
g = 0
size = 3
ground_state = get_ground_state_for_g(g, size)
rad = map_to_radians(g)


trained_disc_weights = tf.Variable(np.array([ 2.86846906e-01,  2.07442139e+02, -1.32505005e+02,  3.37965851e+02,
        1.44702286e+02,  1.67083755e+02,  3.51270142e+02,  4.46218109e+02,
        4.07063354e+02,  1.02569595e+02,  1.62362915e+02,  1.63151154e+02,
        2.18758965e+01, -8.55859451e+01,  1.11934540e+02, -2.94222107e+01,
       -2.17204269e+02,  1.34002686e+02, -1.02429161e+02,  3.34117676e+02,
        1.70663238e-01, -7.37496872e+01,  2.35848450e+02,  5.64170044e+02,
        1.03939911e+02,  1.88091278e+02,  2.74167786e+02,  2.02242035e+02]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([-144.82959 ,  -48.820187,  -37.406292,  -28.891056,   39.797165,
        -91.51027 ,  -34.273388, -168.94511 ,   21.551538,   -5.49935 ,
        -48.562504,   85.00836 ,  -53.48785 ,   34.342926,  -52.16403 ,
        -13.224033, -107.89109 ,   67.52959 ,   66.580956,  -77.641495,
        -80.20973 ,  -68.8333, rad]), dtype=tf.float32)




state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.partial_trace_of_state_vector_as_mixture(state_vector, [0,1,2])

In [29]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

Max prob state has prob: 0.990966260433197


In [30]:
highest_prob_state

array([-0.0319832 +0.j        , -0.15635763-0.02279332j,
       -0.00327316-0.00306216j,  0.07331427-0.02613768j,
        0.03153231-0.12366724j,  0.1965617 -0.5657752j ,
        0.04087236-0.01602049j,  0.26633027+0.7211426j ], dtype=complex64)

In [31]:
get_ground_state_for_g(g, size)

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(g,size)))

1.5697734015047948

In [34]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(g, size))

1.0